In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# navigate to working folder
%cd '/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process'

/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process


In [ ]:
!pwd

/content/drive/MyDrive/UIT/HK1_2023-2024/Project_01/Process


IF YOU HAVE THE DATASET ALREADY, THIS PROCESS CAN BE PASSED BY

In [ ]:
# download dataset .zip from kaggle

# Install Kaggle API client
!pip install -q kaggle

# Upload kaggle API key file using your local file
from google.colab import files
files.upload()

# Set up the Kaggle directory and API key file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Use Kaggle API to download dataset
# You need to replace 'DATASET_PATH' with the path of the dataset you want
!kaggle datasets download -d 'msambare/fer2013'

# For example, for the 'dogs vs cats' dataset, DATASET_PATH would be 'bikashjaiswal/dogs-vs-cats'


In [ ]:
# unzip the dataset

import zipfile

# Unzip the dataset
zip_path = "./dataset/fer2013.zip"  # replace with your downloaded file name
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("./dataset")

THIS IS WHERE THE TRAINING BEGINS

In [ ]:
# import necessary libraries
import os
import cv2
from google.colab import files
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical

In [ ]:
# load img func
def load_images_from_folder(folder):
    images = []
    labels = []
    label_mapping = {
        'angry': 0,
        'disgust': 1,
        'fear': 2,
        'happy': 3,
        'sad': 4,
        'surprise': 5,
        'neutral': 6
    }

    for emotion in os.listdir(folder):
        emotion_folder = os.path.join(folder, emotion)

        for filename in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                images.append(img)
                labels.append(label_mapping[emotion])
    return images, labels

In [ ]:
# load images from train and test
train_images, train_labels = load_images_from_folder('./dataset/emotion_recognition_data/train')
test_images, test_labels = load_images_from_folder('./Process/dataset/emotion_recognition_data/test')

In [ ]:
# type check for comparision reason
print(type(train_images))
print(type(test_images))

<class 'list'>
<class 'list'>


In [ ]:
# convert the images and labels into a DataFrame
def create_dataframe(images, labels, usage):
    pixel_data = [' '.join(map(str, img.flatten())) for img in images]
    data = {'emotion': labels, 'pixels': pixel_data, 'Usage': usage}
    df = pd.DataFrame(data)
    return df

train_df = create_dataframe(train_images, train_labels, 'Training')
test_df = create_dataframe(test_images, test_labels, 'Test')

# Combine the dataframes into one
final_df = pd.concat([train_df, test_df], ignore_index=True)


In [ ]:
# download csv file
final_df.to_csv('fer2013.csv', index=False)
# files.download('fer2013.csv') download if needed

In [ ]:
# Read the dataset into a pandas dataframe
df = pd.read_csv('./fer2013.csv')

In [ ]:
print(df)

       emotion                                             pixels     Usage
0            0  131 133 131 131 132 138 155 150 134 148 141 16...  Training
1            0  118 118 142 165 168 160 165 169 167 168 167 16...  Training
2            0  83 87 88 89 78 96 108 102 119 128 147 162 157 ...  Training
3            0  32 37 45 46 61 67 80 89 89 94 99 107 111 113 1...  Training
4            0  245 227 218 226 221 223 240 235 236 231 225 16...  Training
...        ...                                                ...       ...
35882        5  239 241 240 239 241 238 227 127 124 161 156 15...      Test
35883        5  47 45 51 38 31 34 33 70 32 34 26 26 24 33 14 1...      Test
35884        5  68 79 89 102 117 125 127 127 128 136 140 138 1...      Test
35885        5  228 222 210 194 202 208 209 179 156 143 134 13...      Test
35886        5  46 46 41 48 55 55 62 63 39 33 30 44 45 45 58 5...      Test

[35887 rows x 3 columns]


In [ ]:
# Preprocess data function
def preprocess_data(df):
    images = df["pixels"].apply(lambda x: np.array(x.split(" "), dtype=np.float32))
    images = np.array([x.reshape(48, 48, 1) for x in images])
    images /= 255.0
    labels = to_categorical(df["emotion"])
    return images, labels

In [ ]:
# data splitting and data preprocessing

train_data = df[df["Usage"] == "Training"]
test_data = df[df["Usage"] == "Test"]

x_train, y_train = preprocess_data(train_data)
x_test, y_test = preprocess_data(test_data)

In [ ]:
# build model
model = Sequential([
    Flatten(),
    Dense(7, activation='softmax'),
])

In [ ]:
# compile the model (MUST DO BEFORE TRAINING)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
225/225 [==============================] - 1s 3ms/step - loss: 1.8104 - accuracy: 0.2556
Epoch 2/20
225/225 [==============================] - 1s 3ms/step - loss: 1.7537 - accuracy: 0.2981
Epoch 3/20
225/225 [==============================] - 1s 3ms/step - loss: 1.7288 - accuracy: 0.3145
Epoch 4/20
225/225 [==============================] - 1s 3ms/step - loss: 1.7092 - accuracy: 0.3294
Epoch 5/20
225/225 [==============================] - 1s 3ms/step - loss: 1.6984 - accuracy: 0.3369
Epoch 6/20
225/225 [==============================] - 1s 3ms/step - loss: 1.6888 - accuracy: 0.3426
Epoch 7/20
225/225 [==============================] - 1s 3ms/step - loss: 1.6797 - accuracy: 0.3474
Epoch 8/20
225/225 [==============================] - 1s 3ms/step - loss: 1.6761 - accuracy: 0.3479
Epoch 9/20
225/225 [==============================] - 1s 4ms/step - loss: 1.6716 - accuracy: 0.3478
Epoch 10/20
225/225 [==============================] - 1s 3ms/step - loss: 1.6669 - accuracy: 0.3514

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense_1 (Dense)             (None, 7)                 16135     
                                                                 
Total params: 16135 (63.03 KB)
Trainable params: 16135 (63.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Loss: ", test_loss)
print("Test Accuracy: ", test_acc)

225/225 [==============================] - 1s 2ms/step - loss: 1.7274 - accuracy: 0.3051
Loss:  1.727416753768921
Test Accuracy:  0.30509892106056213


In [ ]:
model_name = 'emotion_model_v01'
# save model
model.save(f"{model_name}.h5")
# Download the model
files.download(f"{model_name}.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>